In [14]:
import os
from PIL import Image
import numpy as np

IMAGE_DIR = "images"
FOLDERS = [f for f in os.listdir(IMAGE_DIR) if "DS_Store" not in f]

In [30]:
images = []

for i, folder in enumerate(FOLDERS):
    fname = os.path.join(IMAGE_DIR, folder)
    for im in os.listdir(fname):
        impath = os.path.join(fname, im)
        img = Image.open(impath)
        data = np.asarray(img, dtype="int32")
        # Scale values from -1 to 1
        data = ((data / 255) - .5) / .5
        data = np.moveaxis(data, -1, 0)
        images.append((data, folder, i))

In [22]:
def log_loss(predicted, actual):
    predicted = predicted / np.sum(predicted, axis=0)
    actual = actual / np.sum(actual, axis=0)
    return np.log(actual) - np.log(predicted)

In [66]:
def init_layers(layer_defs):
    layers = []
    for i in range(1, len(layer_defs)):
        if "input_units" in layer_defs[i]:
            last_units = layer_defs[i]["input_units"]
        else:
            last_units = layer_defs[i-1]["units"]

        biases = np.ones((1,layer_defs[i]["units"]))
        if layer_defs[i]["type"] == "cnn":
            weights = np.random.rand(layer_defs[i-1]["units"], layer_defs[i]["units"], layer_defs[i]["kernel_size"], layer_defs[i]["kernel_size"])
        else:
            weights = np.random.rand(last_units, layer_defs[i]["units"]) / 5 - .1

        layers.append([
            weights,
            biases,
            layer_defs[i]["type"]
        ])
    return layers

In [58]:
def unroll_image(image, kernel_x, kernel_y):
    x_size = (image.shape[0] - (kernel_x - 1))
    y_size = (image.shape[1] - (kernel_y - 1))
    rows =  x_size * y_size
    unrolled = np.zeros((rows, kernel_x * kernel_y))
    for x in range(0, x_size):
        for y in range(0, y_size):
            unrolled[y + (x * y_size),:] = image[x:(x+kernel_x),y:(y+kernel_y)].reshape((1,kernel_x * kernel_y))
    return unrolled

In [70]:
def forward(batch, layers):
    hidden = [batch.copy()]
    for i in range(len(layers)):
        if layers[i][2] == "cnn":
            channels, next_channels, kernel_x, kernel_y = layers[i][0].shape

            new_x = batch.shape[1] - (kernel_x - 1)
            new_y = batch.shape[2] - (kernel_y - 1)
            next_batch = np.zeros((next_channels, new_x , new_y))
            for channel in range(channels):
                unrolled = unroll_image(batch[channel,:], kernel_x, kernel_y)
                for next_channel in range(next_channels):
                    kernel = layers[i][0][channel, next_channel, :].reshape(kernel_x * kernel_y,1)
                    mult = np.matmul(unrolled, kernel).reshape(new_x, new_y)
                    next_batch[next_channel,:] += mult
            next_batch /= batch.shape[0]
            batch = next_batch
        else:
            if layers[i-1][2] == "cnn":
                batch = batch.reshape(batch.shape[0], batch.shape[1] * batch.shape[2])
            batch = np.matmul(batch, layers[i][0]) + layers[i][1]
            hidden.append(batch.copy())
            if i < len(layers) - 1:
                batch = np.maximum(batch, 0)

    return layers, batch, hidden

In [63]:
np.matmul(unroll_image(images[0][0][0,:], 2), np.array([[.5], [.4], [.2], [-.1]])).reshape(255,255)

TypeError: unroll_image() missing 1 required positional argument: 'kernel_y'

In [36]:
forward(images[0][0][0,:3,:3])

array([[-0.04313725, -0.01176471, -0.02745098],
       [-0.18431373, -0.1372549 , -0.05882353],
       [-0.0745098 , -0.1372549 , -0.16078431]])

In [72]:
layer_defs = [
    {"type": "input", "units": 3},
    {"type": "cnn", "kernel_size": 3, "units": 1},
    {"type": "dense", "input_units": 254 * 254, "units": 5}
]

layers = init_layers(layer_defs)

In [73]:
forward(images[0][0], layers)

([[array([[[[0.09503772, 0.87544855, 0.92389415],
            [0.45044165, 0.8539882 , 0.02307905],
            [0.1923502 , 0.68447098, 0.01045441]]],
   
   
          [[[0.15377495, 0.89318302, 0.28731533],
            [0.51132584, 0.14311928, 0.25354344],
            [0.48025753, 0.26038965, 0.6647432 ]]],
   
   
          [[[0.89456913, 0.67264832, 0.29694808],
            [0.42990348, 0.10850559, 0.83958514],
            [0.60568952, 0.83011678, 0.77833648]]]]),
   array([[1.]]),
   'cnn'],
  [array([[-0.0457085 ,  0.00060825, -0.03002156,  0.06810297,  0.06141844],
          [ 0.06844561, -0.00603317, -0.00689269, -0.05096878,  0.05526585],
          [ 0.01379876,  0.01190915,  0.03416909, -0.01606225, -0.06698483],
          ...,
          [-0.08963875, -0.09277641, -0.091673  ,  0.05078235,  0.07784433],
          [ 0.03077603, -0.06850821, -0.07995848,  0.0124015 , -0.00410853],
          [ 0.08598231, -0.08312066,  0.04995632,  0.03172993,  0.09182937]]),
   array([[1., 1.,